Scripts to plot the MCMC-simulated spectra against DSN observations, visualize each rotational transition, and extract the corresponding model frequencies and intensities for spectral line stacking analysis. Bounds for HC<sub>5</sub>N in Cha-MMS1 are included, and others can be added following the same format.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from contextlib import contextmanager

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from spectral_simulator.classes import *
from spectral_simulator.constants import *
from spectral_simulator.functions import *
from inference import *

Load DSN data to extract frequencies and intensities. Ensure you have previously run `inference.py` on your molecule of interest. Otherwise, the required data file will not exist.

In [ ]:
mol_name = 'hc5n_hfs'
 
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
fit_folder = os.path.join(BASE_DIR, 'results')
datafile_path = os.path.join(fit_folder, mol_name, f"all_{mol_name}_lines_DSN_freq_space.npy")
datagrid = np.load(datafile_path, allow_pickle=True)
dsn_freqs = np.array(datagrid[0])
dsn_ints = np.array(datagrid[1])

print(f"Frequencies:\n{dsn_freqs}")
print(f"\nIntensities:\n{dsn_ints}")

Inspect DSN frequencies (above) to manually define frequency bounds for spectral lines of interest. 

In [ ]:
def plot_frequency_range(lower_bound, upper_bound):
    mask = (dsn_freqs >= lower_bound) & (dsn_freqs <= upper_bound)
    filtered_freqs = dsn_freqs[mask]
    filtered_ints = dsn_ints[mask]

    plt.figure(figsize=(10, 4))
    plt.plot(filtered_freqs, filtered_ints, label='Observed Data', color='blue')
    plt.xlabel('Frequency (MHz)')
    plt.ylabel('Intensity')
    plt.title(f'Observed Spectrum from {lower_bound} MHz to {upper_bound} MHz')
    plt.legend()
    plt.show()
    
bounds = [
    (18638.51316476, 18638.69627022),
    (21301.16639020, 21301.34949566),
    (23963.78846446, 23964.00208749),
]

# for lower_bound, upper_bound in bounds:
#     plot_frequency_range(lower_bound, upper_bound)

Define best-fit parameters from MCMC run.

In [ ]:
source_size = 52.00    # arcseconds
Ncol        = 3.24e12  # cm^-2
Tex         = 7.53     # K
dV          = 0.78     # km/s
vlsr        = 4.11     # km/s

To generate model spectrum using best-fit parameters, we use spectral simulator functions used in MCMC.

In [ ]:
aligned_velocity = 4.10  # Aligned velocity (m/s)
dish_size = 70           # Telescope dish diameter (m)
lower_limit = 18000      # Lower frequency limit (MHz)
upper_limit = 25000      # Upper frequency limit (MHz)

# Simulate molecular spectral emission lines
def predict_intensities(Ncol, Tex, dV, mol_cat, source_size):
    obs_params = ObsParams("test", source_size=source_size)
    sim = MolSim("mol sim", mol_cat, obs_params, vlsr=[aligned_velocity], C=[Ncol], dV=[dV], T=[Tex],
                 ll=[lower_limit], ul=[upper_limit], gauss=False)
    freq_sim = np.array(sim.freq_sim)
    int_sim = np.array(sim.int_sim)
    tau_sim = np.array(sim.tau_sim)
    return freq_sim, int_sim, tau_sim

# Apply beam dilution correction
def apply_beam(frequency, intensity, source_size, dish_size):
    wavelength = cm / (frequency * 1e6)
    beam_size = wavelength * 206265 * 1.22 / dish_size  # 206265 converts radians to arcseconds
    dilution_factor = source_size ** 2 / (beam_size ** 2 + source_size ** 2)
    intensity_diluted = intensity * dilution_factor
    return intensity_diluted

# Construct the model spectrum
def make_model(freqs, intensities, source_size, datagrid_freq, datagrid_ints, vlsr, dV, Tex):
    model = np.zeros(datagrid_ints.shape)
    num_lines = freqs.shape[0]

    # Compute Gaussian profiles for each line and sum them
    for i in range(num_lines):
        velocity_grid = (freqs[i] - datagrid_freq) / freqs[i] * ckm + aligned_velocity
        mask = np.abs(velocity_grid - aligned_velocity) < dV * 10

        # Gaussian profile for the intensity at each frequency point
        model[mask] += intensities[i] * np.exp(-0.5 * ((velocity_grid[mask] - vlsr) / (dV / 2.355))**2.)

    # Apply the Planck function for thermal radiation, adjusted for the background cosmic temperature (2.7 K)
    J_T = (h * datagrid_freq * 1e6 / k) / (np.exp((h * datagrid_freq * 1e6) / (k * Tex)) - 1 + 1e-10)
    J_Tbg = (h * datagrid_freq * 1e6 / k) / (np.exp((h * datagrid_freq * 1e6) / (k * 2.7)) - 1 + 1e-10)

    # Apply the beam dilution correction to the model
    model = apply_beam(datagrid_freq, (J_T - J_Tbg) * (1 - np.exp(-model)), source_size, dish_size)
    return model


# Initialize molecular catalog
cat_folder = os.path.join(os.getcwd(), 'CDMS_catalog')
catfile_path = os.path.join(cat_folder, f"{mol_name}.cat")
mol_cat = MolCat(mol_name, catfile_path)

# Generate simulated frequencies and intensities
freqs_model, ints_model, taus_model = predict_intensities(Ncol=Ncol, Tex=Tex, dV=dV,
                                                          mol_cat=mol_cat, source_size=source_size)

# Construct model spectrum
model_spectrum = make_model(freqs=freqs_model, intensities=taus_model, source_size=source_size,
                            datagrid_freq=dsn_freqs, datagrid_ints=dsn_ints,
                            vlsr=vlsr, dV=dV, Tex=Tex)

print(f"Length of dsn_freqs: {len(dsn_freqs)}. Length of model_spectrum: {len(model_spectrum)}")
print(f"Frequencies:\n{dsn_freqs}")
print(f"\nIntensities:\n{dsn_ints}")
print(f"\nModel Spectrum:\n{model_spectrum}")

In [ ]:
output_filename = os.path.join(BASE_DIR, f"{mol_name}_model_intensities.txt")
data_to_save = np.column_stack((dsn_freqs, dsn_ints, model_spectrum))

np.savetxt(output_filename, data_to_save, 
           fmt="%15.8f %15.8f %15.8e",
           header="Frequency(MHz)       Intensity          Model_Spectrum", 
           comments='')

print(f"Data saved to {output_filename}")

Plot the observed data and model spectrum within each of the defined frequency bounds.

In [ ]:
num_bounds = len(bounds)
cols = 3
rows = (num_bounds + cols - 1) // cols
fig, axs = plt.subplots(rows, cols, figsize=(14, 5 * rows))
fig.suptitle(r'HC$_7$N in Cha-MMS1', fontsize=16, weight='bold')

for i, (lower_bound, upper_bound) in enumerate(bounds):
    # Extract observed data within the current bound
    mask = (dsn_freqs >= lower_bound) & (dsn_freqs <= upper_bound)
    filtered_dsn_freqs = dsn_freqs[mask]
    filtered_dsn_ints = dsn_ints[mask]
    
    # Create a finer frequency grid within the current bound
    fine_freqs = np.linspace(lower_bound, upper_bound, num=1000)

    # Compute the model spectrum on the finer frequency grid
    def make_model_fine(freqs, freqs_model, intensities_model, source_size, vlsr, dV, Tex):
        model = np.zeros(freqs.shape)
        num_lines = freqs_model.shape[0]

        for j in range(num_lines):
            velocity_grid = (freqs_model[j] - freqs) / freqs_model[j] * ckm + aligned_velocity
            mask_line = np.abs(velocity_grid - aligned_velocity) < dV * 10

            # Gaussian profile
            model[mask_line] += intensities_model[j] * np.exp(-0.5 * ((velocity_grid[mask_line] - vlsr) / (dV / 2.355))**2.)

        # Apply the Planck function
        J_T = (h * freqs * 1e6 / k) / (np.exp((h * freqs * 1e6) / (k * Tex)) - 1 + 1e-10)
        J_Tbg = (h * freqs * 1e6 / k) / (np.exp((h * freqs * 1e6) / (k * 2.7)) - 1 + 1e-10)

        # Apply the beam dilution
        model = apply_beam(freqs, (J_T - J_Tbg) * (1 - np.exp(-model)), source_size, dish_size)

        return model

    # Compute the model spectrum on the fine grid
    fine_model_ints = make_model_fine(
        freqs=fine_freqs,
        freqs_model=freqs_model,
        intensities_model=taus_model,
        source_size=source_size,
        vlsr=vlsr,
        dV=dV,
        Tex=Tex
    )

    # Determine the subplot axes
    row = i // cols
    col = i % cols
    if rows > 1:
        ax = axs[row, col]
    else:
        ax = axs[col]

    # Plot DSN data (black) and model (colored)
    ax.plot(filtered_dsn_freqs, filtered_dsn_ints, color='black', linewidth=1.5)
    ax.scatter(filtered_dsn_freqs, filtered_dsn_ints, color='black', s=6)
    ax.plot(fine_freqs, fine_model_ints, color='darkmagenta', linestyle='--', linewidth=1.5)

    ax.set_xlabel('Frequency (MHz)')
    ax.set_ylabel('Intensity')
    ax.set_title(f'{lower_bound:.2f}-{upper_bound:.2f} MHz')

# Hide any unused subplots
if num_bounds < rows * cols:
    for j in range(num_bounds, rows * cols):
        if rows > 1:
            fig.delaxes(axs.flatten()[j])
        else:
            fig.delaxes(axs[j])

plt.tight_layout()
plt.savefig(os.path.join(BASE_DIR, f"{mol_name}_model_comparison.png"))
plt.show()